# 📊 공모주 앱 사용자 리뷰 데이터 수집 및 구조화

Google Play 사용자 리뷰를 Selenium 기반으로 자동 수집하고,
NLP 분석을 위한 데이터 구조를 설계하는 과정입니다.

이 Notebook은 이후 사용자 리뷰 NLP 분석 파이프라인의
데이터 소스 생성 단계에 해당합니다.

# 📌 1. 환경 설정 및 라이브러리 로드

In [1]:
import time
import pandas as pd
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 📌 2. 날짜 데이터 정규화 함수

In [2]:
def to_date(review_date):
    review_date = review_date.replace(" ", "").replace("년", "-").replace("월", "-").replace("일", "")
    review_date = datetime.strptime(review_date, "%Y-%m-%d")
    return review_date.date()

# 📌 3. 데이터 소스 구조 설계

In [3]:
apps = dict(공모주일정="com.goodappsoftware.ipo", 공모러="kr.gongmorer", 피너츠="com.daeduck.finuts", 
            일육공="com.qniverse.ipo", 공모주알리미="com.rose.wjtakeginsu")

# 📌 4. Google Play 리뷰 수집 함수

In [4]:
def app_review_extractor(app, days=1095):
    # 웹브라우저를 실행 할 때 사용할 옵션 - 사람인 것처럼 정보 입력
    options = Options()

    # ✅ 브라우저가 꺼지지 않도록 유지
    options.add_experimental_option("detach", True)

    # ✅ 실제 사용자 환경과 유사한 기본 설정
    options.add_argument("--start-maximized")                # 최대화된 창으로 시작
    options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 회피
    options.add_argument("--disable-infobars")               # “Chrome이 자동 테스트 중” 문구 제거
    options.add_argument("--disable-extensions")             # 불필요한 확장 기능 제거
    options.add_argument("--disable-popup-blocking")         # 팝업 차단 비활성화 (사람처럼 행동)
    options.add_argument("--no-sandbox")                     # 보안 샌드박스 비활성화
    options.add_argument("--disable-dev-shm-usage")          # 리눅스 환경 안정성 향상

    # ✅ 실제 브라우저 User-Agent (사람의 브라우저처럼 보이도록)
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/141.0.0.0 Safari/537.36"
    )

    # ✅ 시스템 언어 (정상적인 로케일 표현)
    options.add_argument("--lang=ko-KR")

    # ✅ 자동화 플래그 제거
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    # ✅ WebDriver 감지 우회용 스크립트 (선택적)
    prefs = {"credentials_enable_service": False, "profile.password_manager_enabled": False}
    options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    # 웹브라우저에서 url 열기
    driver.get(f"https://play.google.com/store/apps/details?id={app[1]}")

    # 요소가 실행 가능해질 때 까지 기다리기
    wait = WebDriverWait(driver, 10)

    # 평점 및 리뷰 옆의 -> 버튼이 활성화 되도록 스크롤을 1400px 아래로 내림
    driver.execute_script(f"window.scrollTo({0}, {1400})")
    time.sleep(2)

    # 평점 및 리뷰 옆의 -> 버튼 클릭
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label*="평점 및 리뷰 자세히 알아보기"]')))
    button.click()

    # 최신순으로 리뷰를 정렬하기 위해서 버튼 클릭
    wait.until(EC.element_to_be_clickable((By.ID, "sortBy_1"))).click()
    time.sleep(2)

    # 최신을 찾아 클릭
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))).click()
    time.sleep(2)

    # 스크롤을 내린 후 마지막 자료의 날짜를 찾아서 오늘 날짜부터 1주일 전 날짜와 비교하기

    # 리뷰가 담긴 창을 찾아서 Javascript로 10000px씩 아래로 스크롤
    today = datetime.today()
    end_date = today - timedelta(days=days)
    end_date = end_date.date()
    last_review_date = datetime.today().date()
    end = 1000
    len_list_old = len(driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe"))
    while last_review_date > end_date:
        driver.execute_script(f"document.querySelector('.fysCi.Vk3ZVd').scrollTo(0, {end})")
        time.sleep(2)
        # 리뷰 목록 가져오기 div.RHo1pe
        review_list = driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")
        len_list_new = len(review_list)
        if len_list_old >= len_list_new:
            end += 10000
        else:
            len_list_old = len_list_new


        last_review_date = review_list[-1]
        last_review_date = last_review_date.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute("innerHTML")
        last_review_date = to_date(last_review_date)

        print(len(review_list))
        print(last_review_date, end_date)


    print(len(review_list))
    print(end)

    # 리뷰 목록 가져오기 div.RHo1pe
    # review_list = driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")
    result = {}
    cols = ('리뷰일', '앱이름','별점', '사용자 리뷰','회사 응답')
    for review in review_list:

        # 리뷰일
        review_date = review.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute("innerHTML")
        # 추출한 리뷰일을 날짜형 데이터로 변경
        review_date = to_date(review_date)

        # 별점
        rating = float(review.find_element(By.CSS_SELECTOR, 'div[aria-label*="별표 5개 만점에"]').get_attribute("aria-label").split()[3].replace("개를", ""))

        # 사용자 리뷰
        user_review = review.find_element(By.CSS_SELECTOR, ".h3YV2d").get_attribute("innerHTML")

        # 회사 응답
        try:
            company_reply = review.find_element(By.CSS_SELECTOR, ".ras4vb > div").get_attribute("innerHTML")
        except:
            company_reply = "회사 응답 없음"

        values = (review_date, app[0], rating, user_review, company_reply)
        for key, value in zip(cols, values):
            result.setdefault(key, []).append(value)
    driver.close()    
    df = pd.DataFrame(result)
    return df


# 📌 5. 전체 앱 리뷰 수집 실행

In [5]:
all_results = []

for app in apps.items():
    result = app_review_extractor(app)
    all_results.append(result)
    display(result)

20
2024-12-11 2023-02-20
40
2024-08-11 2023-02-20
60
2024-04-02 2023-02-20
60
2024-04-02 2023-02-20
60
2024-04-02 2023-02-20
80
2024-03-07 2023-02-20
100
2023-02-01 2023-02-20
100
31000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2025-12-25,공모주일정,5.0,요 앱이 얼마나 유용한지는 써보고 실제 활용해 보면 알게 됩니다. 설치한지 1년이 ...,회사 응답 없음
1,2025-12-21,공모주일정,5.0,설치해서 사용하다보니 이 앱이야 말로 제 인생에서 가장 필요한 앱이었다는걸 알게 되...,회사 응답 없음
2,2025-12-21,공모주일정,1.0,광고가 너무길다,회사 응답 없음
3,2025-12-16,공모주일정,4.0,이거 유료버전은 없나요?,회사 응답 없음
4,2025-12-05,공모주일정,5.0,돈버는앱,회사 응답 없음
...,...,...,...,...,...
95,2023-09-18,공모주일정,1.0,로딩이 안돼요 제발 서버관리좀 하세요 #### 아니 뭔가 업데이트를했는지 앱을 실행...,"해당 부분이 개선되고, 달력으로 공모주 일정보기등등 기능도 업그레이드된 최신버전으로..."
96,2023-09-11,공모주일정,5.0,나같은 사람에겐 빛같은 어플이에욥!,회사 응답 없음
97,2023-09-11,공모주일정,1.0,뭔 광고를 적당히 해먹어야지,회사 응답 없음
98,2023-03-09,공모주일정,5.0,잘쓰고 있습니다. 감사합니다. 넘버원님의 블로그 주소가 2023년부터 바뀌었습니다....,회사 응답 없음


20
2025-09-09 2023-02-20
40
2024-08-08 2023-02-20
60
2023-12-07 2023-02-20
60
2023-12-07 2023-02-20
60
2023-12-07 2023-02-20
80
2023-02-22 2023-02-20
100
2022-02-03 2023-02-20
100
31000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2026-01-30,공모러,1.0,안드로이드 업데이트해도 하단바가 자꾸 겹칩니다 버튼이 겹치니깐 입력을 할수가 없어요...,"불편을 드려 죄송합니다. S9, S9+, 노트9 에서 동일 증상이 있다는 연락을 받..."
1,2026-01-06,공모러,5.0,해결! 찬양하라 공모러 갓벽한 공모러,바로 해결되어 다행입니다! 감사합니다! 😆\n\n이 문제는 기기 내 웹뷰 앱의 버그...
2,2025-12-31,공모러,5.0,있으면 좋겠는 거 다 있는 최고의 어플!! 주변에 추천 중입니다!!!,오우 25년 마지막 리뷰 였네요. 좋은 리뷰 &amp; 추천 감사합니다! 26년 새...
3,2025-12-22,공모러,4.0,잘쓰다가 갑자기 어플이 안열립니다. 재설치해도 똑같이 다시시도 표시만나옵니다. 해결...,서비스는 당일 정상화 됐고 근본 원인 파악하여 해결 완료되었습니다! 감사합니다. 😂...
4,2025-12-22,공모러,5.0,오늘 설치했는데 접속이 안됩니다. 다시시도라고 뜨네요,서비스는 당일 정상화 됐고 근본 원인 파악하여 해결 완료되었습니다! 감사합니다. 😂...
...,...,...,...,...,...
95,2022-02-19,공모러,5.0,참좋고 편리한 앱입니다. 강추합니다.,좋은 리뷰 감사해요! 더 편하게 만들게요!
96,2022-02-15,공모러,5.0,"유용한 앱 만들어주셔서 감사합니다. 한 가지 건의 드리고 싶은 게, 수익 계산을 매...","안녕하세요. 리뷰&amp;의견 감사합니다!\n\n일단 매도일을 입력받아야 할텐데요,..."
97,2022-02-09,공모러,5.0,필요한 정보만 심플하게 관리 가능하네요 좋아요~,좋은 리뷰 감사합니다~! ❤
98,2022-02-04,공모러,5.0,굳 좋습니다,리뷰 감사해요~! 😆


20
2024-10-24 2023-02-20
40
2024-02-01 2023-02-20
40
2024-02-01 2023-02-20
60
2023-08-18 2023-02-20
80
2023-07-07 2023-02-20
80
2023-07-07 2023-02-20
100
2023-01-11 2023-02-20
100
31000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2026-01-07,피너츠,5.0,"안녕하세요 아이폰 사용중일때 피너츠 공모주를 잘 사용하였는데, 갤럭시로 핸드폰을 바...","안녕하세요? 공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다. 우선 불..."
1,2025-12-05,피너츠,1.0,갑자기 매매일지서 부터 일정달력 등 뭐든것이 초기화됨. 삭제후 다시깔아도 진행이 안됨.,"안녕하세요? 공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다.\n최근 ..."
2,2025-12-05,피너츠,5.0,503에러 나네요~ 서버 재부팅 ㄱㄱ,"안녕하세요? 공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다.\n최근 ..."
3,2025-12-05,피너츠,1.0,갑자기 로그아웃되더니 로긴도 안되고 둘러보기도 안되고 앱이 맛탱이 갔어요,"안녕하세요? 공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다.\n최근 ..."
4,2025-12-04,피너츠,4.0,최근 핸드폰을 바꿨는데 로그인이 계속 안되네요. 네이버와 카카오 계정 로그인만 나오...,"안녕하세요? 공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다.\n피너츠..."
...,...,...,...,...,...
95,2023-02-13,피너츠,5.0,알림기능이 정말 유용합니다.,"공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다. 공모주 투자자에게 도..."
96,2023-02-09,피너츠,5.0,편하고 유용해요,"공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다. 공모주 투자자에게 도..."
97,2023-02-06,피너츠,5.0,좋은어플 감사합니다. 너무편하네요.,"공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다. 공모주 투자자에게 도..."
98,2023-02-03,피너츠,5.0,대박입니다!,"공모주의 모든 것, 피너츠 공모주를 이용해 주셔서 감사합니다. 공모주 투자자에게 도..."


20
2025-07-28 2023-02-20
40
2024-12-05 2023-02-20
60
2024-06-05 2023-02-20
60
2024-06-05 2023-02-20
80
2024-01-21 2023-02-20
80
2024-01-21 2023-02-20
100
2023-07-15 2023-02-20
115
2022-07-15 2023-02-20
115
31000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2026-01-30,일육공,2.0,오류가 많고 ui ux도 별로임,회사 응답 없음
1,2026-01-30,일육공,5.0,좋아요!!!,회사 응답 없음
2,2026-01-05,일육공,3.0,"어플 자체는 도움이 많이 되는데, 너무 자주 팅기고 로그인도 맨날 풀려서 들어갈 때...",회사 응답 없음
3,2025-12-17,일육공,3.0,일반종목은 투자노트쓰기 버튼이 있던데 스팩종목은 투자노트쓰기 버튼이 없나요?,회사 응답 없음
4,2025-12-11,일육공,2.0,앱 기능이나 인터페이스는 참 좋은데 안드로이드용 앱을 만들었으면 적어도 메인화면에서...,회사 응답 없음
...,...,...,...,...,...
110,2022-08-24,일육공,5.0,심플이즈베스트!! 깔끔해서 가독성 좋고 편리해요~,안녕하세요. 일육공 매니저입니다. 일육공은 단순하면서도 직관적이고 쉬운 서비스가 되...
111,2022-08-24,일육공,5.0,IPO 관련 정보가 한 눈에 들어오도록 구성이 잘 되어 있네요. IPO 흥행 관련 ...,안녕하세요. 일육공 매니저입니다. 저희 일육공 앱에서는 상장일정을 비롯하여 청약증거...
112,2022-08-23,일육공,5.0,상장 일정을 모두 볼수 있어 편리합니다~ 점수로 종목판단을 할수 있는것도 좋습니다,안녕하세요. 일육공 매니저입니다. 저희 일육공 서비스는 일반종목의 상장일정을 비롯하...
113,2022-08-22,일육공,5.0,자문점수가 잘 나와서 점수만 보고 따라가면 되서 쉽습니다,안녕하세요. 일육공 매니저입니다. 저희 일육공 서비스는 투자자분들의 고민을 덜어드릴...


20
2024-11-23 2023-02-20
35
2022-12-01 2023-02-20
35
11000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2025-06-04,공모주알리미,1.0,2025.6.24~25일 청약인 기업을 4~5일 청약으로 알려주는 등의 오정보가 하...,"안녕하세요, 먼저 저희 앱을 이용해주셔서 진심으로 감사드립니다. 불편을 겪게 해드려..."
1,2025-03-20,공모주알리미,5.0,청약공모주 알아보는데 정보가 많다보니 비교해본다고 네이버에서 매번 검색하고 했는데 ...,"안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."
2,2025-03-19,공모주알리미,5.0,청약저축과 비슷한 것 같았는데 꽤 많이 나와있어 잘 알아볼 수가 있었고 이 앱을 이...,"안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."
3,2025-03-13,공모주알리미,5.0,신규상장 종목과 일정 공모주에 대한 여러가지 정보가 많이 있어서 관심 있는 분들은 ...,"안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."
4,2025-03-13,공모주알리미,5.0,공모주 청약에 대해 정보가 필요했는데 이 어플 통해서 많은걸 알게되어 부담없이 해볼...,"안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."
5,2025-03-11,공모주알리미,5.0,"공모주 알리미 어플을 통해서 청약, IPO 등 많은 정보를 얻을 수 있어서 편리하고...","안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."
6,2025-03-11,공모주알리미,5.0,"공모주 알리미 앱은 최신 공모주 일정과 관련 정보를 실시간으로 제공하므로, 공모주가...","안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."
7,2025-03-10,공모주알리미,5.0,"공모주 청약 일정과 IPO 정보를 한눈에 확인할 수 있어 유용하며, 경쟁률 분석까지...","안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."
8,2025-03-09,공모주알리미,5.0,공모주 청약이 처음이라 막막했었는데 이 어플 덕분에 많은 정보를 얻을 수 있었네요....,"안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."
9,2025-03-08,공모주알리미,5.0,"공린이도 도전! 공모주 청약에 대해서 자세하게 설명되어 있어요. 공모주정보, 일정,...","안녕하세요! 공모주 알리미 - 청약, IPO, 경쟁률 앱을 사용해 주셔서 진심으로 ..."


# 📌6. 데이터 통합 및 저장

In [6]:
final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv("all_review.csv", index=False, encoding="utf-8-sig")

## ✔️ 다음 단계

본 Notebook은 리뷰 데이터 수집 및 구조화 단계이며,
이후 Notebook에서는 다음 분석이 이어진다.

- 감성 분석 기반 긍·부정 분포 분석
- LDA 토픽 모델링
- Word2Vec 의미 구조 분석